In [4]:
pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 9.7 MB/s eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.7/157.7 kB 8.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Imports 

import requests
import pandas as pd
import json

In [ ]:

import requests
import pandas as pd

# PART 1: 1.EXTRACT
# API endpoint for GeoJSON
api_url = "https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"


#took help from AI here to figure out try and except feature to handle exceptions
try:
    # Send request to API
    response = requests.get(api_url)
    response.raise_for_status()  # Uncomment for better error reporting
    data = response.json()

    # Extract features
    features = data.get("features", [])

    # Create records with properties and coordinates
    records = []
    for feature in features:
        props = feature["properties"]
        coords = feature["geometry"]["coordinates"]
        props["longitude"] = coords[0]
        props["latitude"] = coords[1]
        records.append(props)

    # Convert to DataFrame
    df = pd.DataFrame(records)
    print(df.head())

except requests.exceptions.RequestException as e:
    print("API request failed:", e)

#converted the data to CSV to open in excel to understand the Data, what are the coloums and what this Data means 
df. to_csv("air_quality_data.csv")

   OBJECTID siteNo           location  SiteID  Easting  Northing Current_  \
0         1      1   Withywood School       1   356434    167823       No   
1         2     B1     Colston Avenue       2   358628    173011      Yes   
2         3    B10      Blackboy Hill       3   357448    174650      Yes   
3         4    B11        Three Lamps       4   359903    171850      Yes   
4         5    B12  Bedminster Parade       5   358723    171704      Yes   

  pollutants  InstrumentID Exposure  ...  DateStart  DateEnd Colocated  \
0    BTX NO2             1       No  ...        NaN      NaN       NaN   
1        NO2             1       No  ...        NaN      NaN       NaN   
2        NO2             1      Yes  ...        NaN      NaN       NaN   
3        NO2             1       No  ...        NaN      NaN       NaN   
4        NO2             1      Yes  ...        NaN      NaN       NaN   

  Duplicate_Triplicate obstruction_m  obstruction_lt  inlet_flow_clear  \
0                 

In [ ]:
#Part 1 : 2.TRANSFORM 

# Normalize the Data - define the normalize function 
# and then use it to normalise data 
#In anticipation of the loaded data eventually being used by Data Scientists and Analysts,
# the coordinates of each record need to be normalised to values between 0 and 1, this will
# mitigate the latitude values taking precedence over the longitude values in potential
# machine learning applications.
# Create a function from first principles that normalises a given value between 0 and 1.
# Write at least two assertion tests for this function to ensure it works as intended before
# you run the full application.

# why are we Normalizing the Latitudes & longitudes? 
# Normalization rescales values (like latitude and longitude) to a 0–1 range so that no variable dominates due to larger raw numbers, ensuring fair comparison in analysis or machine learning.

def normalise (value, min_value, max_value):
    if max_value == min_value:
        raise ValueError("min_value and max_value cannot be the same.")
    return (value - min_value) / (max_value - min_value)


